DataFrame

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
from matplotlib import pyplot as plt 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_results=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fifa/results.csv")


In [ ]:
# FIFA 월드컵 2022년도 제거 
df_results = df_results.loc[df_results['date']>='2014-01-01'].loc[df_results['date']<'2022-12-31']

In [ ]:

df_results.dropna(axis=0)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
36109,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True
36110,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True
36111,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False
36112,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False
36113,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False
...,...,...,...,...,...,...,...,...,...
44197,2022-11-30,Tunisia,France,1.0,0.0,FIFA World Cup,Al Rayyan,Qatar,True
44198,2022-12-01,Japan,Spain,2.0,1.0,FIFA World Cup,Al Rayyan,Qatar,True
44199,2022-12-01,Costa Rica,Germany,2.0,4.0,FIFA World Cup,Al Khor,Qatar,True
44200,2022-12-01,Croatia,Belgium,0.0,0.0,FIFA World Cup,Al Rayyan,Qatar,True


In [ ]:
# result table, ranking table 을 year 기준으로 join 하기 
df_results['date'] = pd.to_datetime(df_results['date'])
df_results['year'] = df_results['date'].dt.year
df_results.reset_index(drop=True)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
0,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True,2014
1,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True,2014
2,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False,2014
3,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False,2014
4,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False,2014
...,...,...,...,...,...,...,...,...,...,...
8092,2022-12-01,Canada,Morocco,1.0,2.0,FIFA World Cup,Doha,Qatar,True,2022
8093,2022-12-02,Serbia,Switzerland,NaN,NaN,FIFA World Cup,Doha,Qatar,True,2022
8094,2022-12-02,Cameroon,Brazil,NaN,NaN,FIFA World Cup,Lusail,Qatar,True,2022
8095,2022-12-02,Ghana,Uruguay,NaN,NaN,FIFA World Cup,Al Wakrah,Qatar,True,2022


In [ ]:
df_results = df_results.drop(df_results[(df_results['year'] == 2022) & (df_results['tournament'] == 'FIFA World Cup')].index)
# 2022 fifa world cup 경기 결과제거 
df_results.shape[0]

8049

In [ ]:
df_rankings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fifa/fifa_ranking-2022-10-06.csv')
df_rankings.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31


In [ ]:
df_rankings = df_rankings.replace({"IR Iran":"Iran"})
df_rankings = df_rankings.replace({"USA" : "United States"})
df_rankings = df_rankings.replace({"Korea Republic" : "South Korea"})
df_rankings = df_rankings.replace({"Korea DPR" : "South Korea"})

df_rankings = df_rankings.loc[df_rankings['rank_date']>='2014-01-01'].loc[df_rankings['rank_date'] <='2022-12-31']

In [ ]:
df_rankings = pd.DataFrame(df_rankings, columns = ['country_full', 'total_points', 'rank_date']).reset_index(drop=True)
df_rankings.shape[0]

18464

In [ ]:
df_rankings['rank_date'] = pd.to_datetime(df_rankings['rank_date'])
df_rankings['year'] = df_rankings['rank_date'].dt.year
df_rankings.head()

,country_full,total_points,rank_date,year
0,Cape Verde Islands,726.0,2014-01-16,2014
1,Turkey,677.0,2014-01-16,2014
2,Tunisia,689.0,2014-01-16,2014
3,Honduras,692.0,2014-01-16,2014
4,Peru,698.0,2014-01-16,2014


In [ ]:
df_rankings.sort_values(by=['rank_date','year'],inplace=True, ascending=False)
latest_rankdate_index = df_rankings[['country_full', 'year']].drop_duplicates().index
df_rankings = df_rankings.loc[lambda x : x.index.isin(latest_rankdate_index)]

In [ ]:
dataset = df_results
dataset['home_team_total_points'] = np.nan
dataset['away_team_total_points'] = np.nan

In [ ]:
for index in dataset.index:
  team1 = dataset.loc[index, ['home_team']]
  team2 = dataset.loc[index, ['away_team']]
  year = dataset.loc[index, ['year']]
  year = int(year)
  team1 = team1[0]
  team2 = team2[0]
  try:
    data1 = df_rankings['total_points'].loc[(df_rankings['country_full'] == team1) & (df_rankings['year'] == year)]
    data2 = df_rankings['total_points'].loc[(df_rankings['country_full'] == team2) & (df_rankings['year'] == year)]
    dataset.loc[index, ['home_team_total_points']] = float(data1)
    dataset.loc[index, ['away_team_total_points']] = float(data2)
  except:
    continue


In [ ]:
dataset.shape[0]

8049

In [ ]:
dataset.isnull().sum()

date                         0
home_team                    0
away_team                    0
home_score                   0
away_score                   0
tournament                   0
city                         0
country                      0
neutral                      0
year                         0
home_team_total_points     723
away_team_total_points    1172
dtype: int64

In [ ]:
dataset = dataset.dropna(axis = 0)

In [ ]:
dataset.isnull().sum()

date                      0
home_team                 0
away_team                 0
home_score                0
away_score                0
tournament                0
city                      0
country                   0
neutral                   0
year                      0
home_team_total_points    0
away_team_total_points    0
dtype: int64

In [ ]:
dataset.shape[0]

6877

player 14 ~ 23년도

In [ ]:
# home team 
# 수비수 4명
for i in range(1, 5):
  col = 'home_B'+str(i)+'_point'
  dataset[col] = np.nan

# 미드필더 4명 
for i in range(1, 5):
  col = 'home_M'+str(i)+'_point'
  dataset[col] = np.nan

# 공격수 2명 
for i in range(1, 3):
  col = 'home_T'+str(i)+'_point'
  dataset[col] = np.nan

# 골키퍼 1명 
col = 'home_K1_point'
dataset[col] = np.nan

<ipython-input-138-141b4fe05a4a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[col] = np.nan
<ipython-input-138-141b4fe05a4a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[col] = np.nan
<ipython-input-138-141b4fe05a4a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [ ]:
# away team 
# 수비수 4명
for i in range(1, 5):
  col = 'away_B'+str(i)+'_point'
  dataset[col] = np.nan

# 미드필더 4명 
for i in range(1, 5):
  col = 'away_M'+str(i)+'_point'
  dataset[col] = np.nan

# 공격수 2명 
for i in range(1, 3):
  col = 'away_T'+str(i)+'_point'
  dataset[col] = np.nan

# 골키퍼 1명 
col = 'away_K1_point'
dataset[col] = np.nan

In [ ]:
# value 
# home team 
# 수비수 4명
for i in range(1, 5):
  col = 'home_B'+str(i)+'_value'
  dataset[col] = np.nan

# 미드필더 4명 
for i in range(1, 5):
  col = 'home_M'+str(i)+'_value'
  dataset[col] = np.nan

# 공격수 2명 
for i in range(1, 3):
  col = 'home_T'+str(i)+'_value'
  dataset[col] = np.nan

# 골키퍼 1명 
col = 'home_K1_value'
dataset[col] = np.nan

<ipython-input-140-2a44d816887a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[col] = np.nan
<ipython-input-140-2a44d816887a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[col] = np.nan
<ipython-input-140-2a44d816887a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [ ]:
# away team 
# 수비수 4명
for i in range(1, 5):
  col = 'away_B'+str(i)+'_value'
  dataset[col] = np.nan

# 미드필더 4명 
for i in range(1, 5):
  col = 'away_M'+str(i)+'_value'
  dataset[col] = np.nan

# 공격수 2명 
for i in range(1, 3):
  col = 'away_T'+str(i)+'_value'
  dataset[col] = np.nan

# 골키퍼 1명 
col = 'away_K1_value'
dataset[col] = np.nan

<ipython-input-88-aa66df6b8fbe>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[col] = np.nan
<ipython-input-88-aa66df6b8fbe>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[col] = np.nan
<ipython-input-88-aa66df6b8fbe>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [ ]:
dataset.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,...,home_B2_value,home_B3_value,home_B4_value,home_M1_value,home_M2_value,home_M3_value,home_M4_value,home_T1_value,home_T2_value,home_K1_value
36109,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36110,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36111,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36112,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36113,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


최종 dataset 합치는 작업

In [ ]:
# FIFA 15-23 player 최종본 
df_player14 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=0)
df_player15 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=1)
df_player16 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=2)
df_player17 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=3)
df_player18 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=4)
df_player19 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=5)
df_player20 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=6)
df_player21 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=7)
df_player22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=8)

In [ ]:
df_player14['year'] = 2014
df_player15['year'] = 2015
df_player16['year'] = 2016
df_player17['year'] = 2017
df_player18['year'] = 2018
df_player19['year'] = 2019
df_player20['year'] = 2020
df_player21['year'] = 2021
df_player22['year'] = 2022

In [ ]:
df_player = pd.concat([df_player14, df_player15, df_player16, df_player17, df_player18, df_player19, df_player20, df_player21, df_player22], ignore_index=True)

In [ ]:
# nationality_name, Model_Position, year 로 group by 한 후, Position_Best_Rating 에 대해 rank 매기기 
df_player['rank_in_nation'] = df_player.groupby(['nationality_name','Model_Position', 'year'])['Position_Best_Rating'].rank(method='first',ascending=False)

In [ ]:
df_player = df_player.astype({'rank_in_nation':'int'})
df_player.head()

,short_name,overall,value_eur,wage_eur,age,height_cm,weight_kg,nationality_name,preferred_foot,weak_foot,...,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,Position_Best_Rating,Model_Position,year,rank_in_nation
0,A. Meha,70,1600000,25000,28,174,76,Albania,Right,4,...,47,9,12,14,8,13,70,M,2014,3
1,E. Berisha,72,2500000,25000,25,194,92,Albania,Left,1,...,25,76,71,64,71,76,72,K,2014,1
2,E. Hysaj,67,500000,7000,20,182,70,Albania,Left,3,...,67,11,5,13,8,10,67,B,2014,3
3,E. Kace,69,825000,7000,20,161,58,Albania,Right,4,...,63,6,15,14,7,11,69,M,2014,4
4,J. Hyka,70,1800000,25000,26,169,62,Albania,Right,3,...,30,8,9,13,9,7,72,T,2014,2


In [ ]:
df_player = df_player.replace({"Korea Republic":"South Korea"})

In [ ]:
df_player.loc[df_player['nationality_name'].str.contains('Korea')] # 값이 같은 경우 rank 

,short_name,overall,value_eur,wage_eur,age,height_cm,weight_kg,nationality_name,preferred_foot,weak_foot,...,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,Position_Best_Rating,Model_Position,year,rank_in_nation
440,H. Son,76,4500000,70000,21,183,76,South Korea,Left,5,...,22,10,12,12,5,9,77,T,2014,2
441,Hong Chul,69,800000,15000,23,176,69,South Korea,Left,3,...,64,10,10,12,6,14,70,B,2014,3
442,Hong Jeong Ho,71,1500000,20000,24,186,77,South Korea,Right,4,...,68,5,8,13,10,9,71,B,2014,2
443,Ki Sung Yueng,74,3800000,35000,25,186,75,South Korea,Right,4,...,67,13,8,6,8,9,75,M,2014,2
444,Kim Do Heon,72,2200000,30000,31,177,73,South Korea,Right,4,...,57,7,12,12,13,14,72,M,2014,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7607,Jo Hyeon Woo,73,2000000,8000,30,189,75,South Korea,Right,3,...,16,73,70,62,70,78,73,K,2022,1
7608,Kim Min Jae,79,23000000,49000,25,190,81,South Korea,Right,3,...,77,13,7,11,14,11,77,M,2022,1
7609,Kim Tae Hwan,72,1600000,8000,32,177,72,South Korea,Right,3,...,68,12,10,16,9,11,72,B,2022,4
7610,Lee Jae Sung,76,7500000,28000,29,181,74,South Korea,Left,3,...,54,14,7,15,6,8,76,M,2022,3


In [ ]:
df_player.loc[df_player['nationality_name'].str.contains('United')]

,short_name,overall,value_eur,wage_eur,age,height_cm,weight_kg,nationality_name,preferred_foot,weak_foot,...,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,Position_Best_Rating,Model_Position,year,rank_in_nation
781,C. Dempsey,81,10000000,70000,31,185,84,United States,Right,4,...,47,12,7,5,10,12,81,T,2014,1
782,F. Johnson,74,3500000,45000,26,183,74,United States,Right,5,...,73,6,6,9,11,14,76,M,2014,3
783,G. Cameron,73,2200000,40000,28,191,84,United States,Right,3,...,71,6,15,5,9,13,73,B,2014,1
784,J. Jones,77,2800000,50000,32,184,78,United States,Right,4,...,77,11,7,5,10,5,77,M,2014,2
785,L. Donovan,79,5500000,100000,32,173,73,United States,Right,4,...,32,10,7,9,8,15,79,T,2014,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8025,S. Dest,77,22500000,71000,21,175,62,United States,Right,4,...,74,11,10,10,15,15,79,M,2022,2
8026,T. Adams,76,10000000,29000,23,175,72,United States,Right,3,...,70,6,13,12,8,7,77,M,2022,4
8027,W. McKennie,80,31500000,90000,23,183,84,United States,Right,3,...,75,7,9,12,9,11,81,T,2022,2
8028,W. Zimmerman,76,7000000,9000,29,191,89,United States,Right,3,...,68,13,14,8,10,9,77,B,2022,2


In [ ]:
dataset.loc[dataset['away_team'].str.contains('United States')]

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,...,home_B2_value,home_B3_value,home_B4_value,home_M1_value,home_M2_value,home_M3_value,home_M4_value,home_T1_value,home_T2_value,home_K1_value
36219,2014-03-05,Ukraine,United States,2.0,0.0,Friendly,Larnaca,Cyprus,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36437,2014-06-16,Ghana,United States,1.0,2.0,FIFA World Cup,Natal,Brazil,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36485,2014-07-01,Belgium,United States,2.0,1.0,FIFA World Cup,Salvador,Brazil,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36542,2014-09-03,Czech Republic,United States,0.0,1.0,Friendly,Prague,Czech Republic,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36901,2014-11-18,Republic of Ireland,United States,4.0,1.0,Friendly,Dublin,Republic of Ireland,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37043,2015-01-28,Chile,United States,3.0,2.0,Friendly,Rancagua,Chile,False,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37081,2015-03-25,Denmark,United States,3.0,2.0,Friendly,Aarhus,Denmark,False,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37183,2015-03-31,Switzerland,United States,1.0,1.0,Friendly,Zürich,Switzerland,False,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37238,2015-06-05,Netherlands,United States,3.0,4.0,Friendly,Amsterdam,Netherlands,False,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37275,2015-06-10,Germany,United States,1.0,2.0,Friendly,Cologne,Germany,False,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for index in dataset.index:
  team1 = dataset.loc[index, ['home_team']]
  team2 = dataset.loc[index, ['away_team']]
  
  year = dataset.loc[index, ['year']]
  year = int(year)
  team1 = team1[0]
  team2 = team2[0]
  # position B (수비수) 4명 
  for i in range(1, 5):
    col1 = 'home_B'+str(i)+'_point'
    col2 = 'home_B'+str(i)+'_value'
    col3 = 'away_B'+str(i)+'_point'
    col4 = 'away_B'+str(i)+'_value'
    try:
      data = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'B') & (df_player['nationality_name'] == team1 ) & (df_player['year'] == year)]
      dataset.loc[index, [col1]] = float(data)
      data2 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'B') & (df_player['nationality_name'] == team1) & (df_player['year'] == year)]
      dataset.loc[index, [col2]] = float(data2)
      data3 = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'B') & (df_player['nationality_name'] == team2 ) & (df_player['year'] == year)]
      dataset.loc[index, [col3]] = float(data3)
      data4 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'B') & (df_player['nationality_name'] == team2) & (df_player['year'] == year)]
      dataset.loc[index, [col4]] = float(data4)
    except:
      continue

  # position M (미드필더) 4명 
  for i in range(1, 5):
    col1 = 'home_M'+str(i)+'_point'
    col2 = 'home_M'+str(i)+'_value'
    col3 = 'away_M'+str(i)+'_point'
    col4 = 'away_M'+str(i)+'_value'
    try:
      data = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'M') & (df_player['nationality_name'] == team1 ) & (df_player['year'] == year)]
      dataset.loc[index, [col1]] = float(data)

      data2 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'M') & (df_player['nationality_name'] == team1) & (df_player['year'] == year)]
      dataset.loc[index, [col2]] = float(data2)

      data3 = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'M') & (df_player['nationality_name'] == team2 ) & (df_player['year'] == year)]
      dataset.loc[index, [col3]] = float(data3)

      data4 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'M') & (df_player['nationality_name'] == team2) & (df_player['year'] == year)]
      dataset.loc[index, [col4]] = float(data4)
    except:
      continue
  # position (공격수) 2명 -> T
  for i in range(1, 3):
    col1 = 'home_T'+str(i)+'_point'
    col2 = 'home_T'+str(i)+'_value'
    col3 = 'away_T'+str(i)+'_point'
    col4 = 'away_T'+str(i)+'_value'
    try:
      data = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'T') & (df_player['nationality_name'] == team1 ) & (df_player['year'] == year)]
      dataset.loc[index, [col1]] = float(data)

      data2 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'T') & (df_player['nationality_name'] == team1) & (df_player['year'] == year)]
      dataset.loc[index, [col2]] = float(data2)

      data3 = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'T') & (df_player['nationality_name'] == team2 ) & (df_player['year'] == year)]
      dataset.loc[index, [col3]] = float(data3)

      data4 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'T') & (df_player['nationality_name'] == team2) & (df_player['year'] == year)]
      dataset.loc[index, [col4]] = float(data4)
    except:
      continue

  # # 골키퍼 
  col1 = 'home_K1_point'
  col2 = 'home_K1_value'
  col3 = 'away_K1_point'
  col4 = 'away_K1_value'
  try:
    data = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == 1) & (df_player['Model_Position'] == 'K') & (df_player['nationality_name'] == team1 ) & (df_player['year'] == year)]
    dataset.loc[index, [col1]] = float(data)
    data2 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == 1) & (df_player['Model_Position'] == 'K') & (df_player['nationality_name'] == team1) & (df_player['year'] == year)]
    dataset.loc[index, [col2]] = float(data2)
    data3 = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == 1) & (df_player['Model_Position'] == 'K') & (df_player['nationality_name'] == team2 ) & (df_player['year'] == year)]
    dataset.loc[index, [col3]] = float(data3)
    data4 =df_player['value_eur'].loc[(df_player['rank_in_nation'] == 1) & (df_player['Model_Position'] == 'K') & (df_player['nationality_name'] == team2) & (df_player['year'] == year)]
    dataset.loc[index, [col4]] = float(data4)
  except:
    continue

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:

for index in dataset.index:
  team1 = dataset.loc[index, ['home_team']]
  team2 = dataset.loc[index, ['away_team']]
  
  year = dataset.loc[index, ['year']]
  year = int(year)
  team1 = team1[0]
  team2 = team2[0]
  # position B (수비수) 4명 
  for i in range(1, 5):
    col1 = 'home_B'+str(i)+'_point'
    col2 = 'home_B'+str(i)+'_value'
    col3 = 'away_B'+str(i)+'_point'
    col4 = 'away_B'+str(i)+'_value'
    try:
      # data1 = df_rankings['total_points'].loc[(df_rankings['country_full'] == team1) & (df_rankings['year'] == year)]
      data = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'B') & (df_player['nationality_name'] == team1 ) & (df_player['year'] == year)]
      dataset.loc[index, [col1]] = float(data)
    except:
      continue
    try:
      data2 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'B') & (df_player['nationality_name'] == team1) & (df_player['year'] == year)]
      dataset.loc[index, [col2]] = float(data2)
    except:
      continue
    try:
      data3 = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'B') & (df_player['nationality_name'] == team2 ) & (df_player['year'] == year)]
      #df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'B'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      dataset.loc[index, [col3]] = float(data3)
    except:
      continue
    try:
      data4 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'B') & (df_player['nationality_name'] == team2) & (df_player['year'] == year)]
      
      
      #df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'B'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      dataset.loc[index, [col4]] = float(data4)
    except:
      continue

    
  # position M (미드필더) 4명 
  for i in range(1, 5):
    col1 = 'home_M'+str(i)+'_point'
    col2 = 'home_M'+str(i)+'_value'
    col3 = 'away_M'+str(i)+'_point'
    col4 = 'away_M'+str(i)+'_value'
    try:
      data = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'M') & (df_player['nationality_name'] == team1 ) & (df_player['year'] == year)]
      #df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'M'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      dataset.loc[index, [col1]] = float(data)
    except:
      continue
    try:
      
      data2 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'M') & (df_player['nationality_name'] == team1) & (df_player['year'] == year)]
      #df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'M'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      dataset.loc[index, [col2]] = float(data2)
    except:
      continue
    try:
      data3 = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'M') & (df_player['nationality_name'] == team2 ) & (df_player['year'] == year)]
      #df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'M'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      dataset.loc[index, [col3]] = float(data3)
    except:
      continue
    try:
      data4 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'M') & (df_player['nationality_name'] == team2) & (df_player['year'] == year)]
      #df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'M'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      dataset.loc[index, [col4]] = float(data4)
    except:
      continue
  # position (공격수) 2명 -> T
  for i in range(1, 3):
    col1 = 'home_T'+str(i)+'_point'
    col2 = 'home_T'+str(i)+'_value'
    col3 = 'away_T'+str(i)+'_point'
    col4 = 'away_T'+str(i)+'_value'
    try:
      data = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'T') & (df_player['nationality_name'] == team1 ) & (df_player['year'] == year)]
      #df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'T'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      dataset.loc[index, [col1]] = float(data)
    except:
      continue
    try:
      data2 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'T') & (df_player['nationality_name'] == team1) & (df_player['year'] == year)]
      #df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'T'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      dataset.loc[index, [col2]] = float(data2)
    except:
      continue
    try:
      data3 = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'T') & (df_player['nationality_name'] == team2 ) & (df_player['year'] == year)]
      #df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'T'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      dataset.loc[index, [col3]] = float(data3)
    except:
      continue
    try:
      data4 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == i) & (df_player['Model_Position'] == 'T') & (df_player['nationality_name'] == team2) & (df_player['year'] == year)]
      #df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'T'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      dataset.loc[index, [col4]] = float(data4)
    except:
      continue

  # # 골키퍼 
  col1 = 'home_K1_point'
  col2 = 'home_K1_value'
  col3 = 'away_K1_point'
  col4 = 'away_K1_value'
  try:
    data = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == 1) & (df_player['Model_Position'] == 'K') & (df_player['nationality_name'] == team1 ) & (df_player['year'] == year)]
    dataset.loc[index, [col1]] = float(data)
  except:
    continue
  try:
    data2 = df_player['value_eur'].loc[(df_player['rank_in_nation'] == 1) & (df_player['Model_Position'] == 'K') & (df_player['nationality_name'] == team1) & (df_player['year'] == year)]
    dataset.loc[index, [col2]] = float(data2)
  except:
    continue
  try:
    data3 = df_player['Position_Best_Rating'].loc[(df_player['rank_in_nation'] == 1) & (df_player['Model_Position'] == 'K') & (df_player['nationality_name'] == team2 ) & (df_player['year'] == year)]
    dataset.loc[index, [col3]] = float(data3)
  except:
    continue
  try:
    data4 =df_player['value_eur'].loc[(df_player['rank_in_nation'] == 1) & (df_player['Model_Position'] == 'K') & (df_player['nationality_name'] == team2) & (df_player['year'] == year)]
    dataset.loc[index, [col4]] = float(data4)
  except:
    continue

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
dataset = dataset.drop(['date', 'country', 'city'], axis=1)

In [ ]:
dataset.isnull().sum()

home_team                    0
away_team                    0
home_score                   0
away_score                   0
tournament                   0
neutral                      0
year                         0
home_team_total_points       0
away_team_total_points       0
home_B1_point             3188
home_B2_point             3188
home_B3_point             3188
home_B4_point             3188
home_M1_point             3188
home_M2_point             3188
home_M3_point             3188
home_M4_point             3188
home_T1_point             3188
home_T2_point             3188
home_K1_point             3188
away_B1_point             4329
away_B2_point             4329
away_B3_point             4329
away_B4_point             4329
away_M1_point             4329
away_M2_point             4329
away_M3_point             4329
away_M4_point             4329
away_T1_point             4329
away_T2_point             4329
away_K1_point             4329
home_B1_value             3188
home_B2_

In [ ]:
new_dataset = dataset.iloc[:,0:34] # value 뺐을 떄 
#new_dataset = new_dataset.dropna(axis=0)

In [ ]:
new_dataset = new_dataset.dropna(axis=0)

In [ ]:
new_dataset.shape[0]

2548

In [ ]:
new_dataset.to_csv('fifa_dataset_without_value.csv') # 최종 dataset 출력 

In [ ]:
new_dataset.isnull().sum()

home_team                 0
away_team                 0
home_score                0
away_score                0
tournament                0
neutral                   0
year                      0
home_team_total_points    0
away_team_total_points    0
home_B1_point             0
home_B2_point             0
home_B3_point             0
home_B4_point             0
home_M1_point             0
home_M2_point             0
home_M3_point             0
home_M4_point             0
home_T1_point             0
home_T2_point             0
home_K1_point             0
home_B1_value             0
home_B2_value             0
home_B3_value             0
home_B4_value             0
home_M1_value             0
home_M2_value             0
home_M3_value             0
home_M4_value             0
home_T1_value             0
home_T2_value             0
home_K1_value             0
away_B1_value             0
away_B2_value             0
away_B3_value             0
dtype: int64

In [ ]:
dataset_copy = dataset
dataset_copy = dataset_copy.dropna(axis=0)

In [ ]:
dataset_copy.shape[0]

2548

In [ ]:
dataset_copy.to_csv('fifa_dataset.csv') # 최종 dataset 출력 (value 결측값 뺀 것)